In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [3]:
from textblob import TextBlob
from textblob import Word
import nltk
from nltk.corpus import stopwords
from stop_words import safe_get_stop_words
from stop_words import get_stop_words

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [5]:
TRAIN_PATH_LBL = './train/labels.json'
TRAIN_PATH_TXT = './train/texts/'
TEST_PATH_LBL = './test/labels.json'
TEST_PATH_TXT = './test/texts/'

In [6]:
# 0 - Tartak, 1 - Elza's Ocean
df_names = ['dict']
train = pd.DataFrame(pd.read_json(TRAIN_PATH_LBL, typ='series'), columns=df_names).reset_index()
train.columns = ['song', 'dict'] 
test = pd.DataFrame(pd.read_json(TEST_PATH_LBL, typ='series'), columns=df_names).reset_index()
test.columns = ['song', 'dict']

y_true = [0 if 'Тартак' in band.values() else 1 for band in train['dict']]
train['band'] = pd.Series(y_true)

y_test = [0 if 'Тартак' in band.values() else 1 for band in test['dict']]
test['band'] = pd.Series(y_test)

In [7]:
train.head()

,song,dict,band
0,001.txt,"{'artist': 'Тартак', 'title': '100-ий плагіат'}",0
1,002.txt,"{'artist': 'Тартак', 'title': '3Dенс'}",0
2,003.txt,"{'artist': 'Тартак', 'title': 'DJ Вовочка'}",0
3,004.txt,"{'artist': 'Тартак', 'title': 'Mikpoffonha пер...",0
4,005.txt,"{'artist': 'Тартак', 'title': 'Qарпа-ратів'}",0


In [8]:
text_series = pd.DataFrame()
text_test_series = pd.DataFrame()

for song in train['song']:
    content = ''
    with open(TRAIN_PATH_TXT + song, encoding='utf-8') as f:
        line = f.read().replace('\n', ' ')
    text_series = text_series.append(pd.Series(line), ignore_index=True)
    
for s in test['song']:
    content = ''
    with open(TEST_PATH_TXT + s, encoding='utf-8') as f:
        line = f.read().replace('\n', ' ')
    text_test_series = text_test_series.append(pd.Series(line), ignore_index=True)

In [9]:
text_series.head()

,0
0,Дивлюсь на себе в дзеркало й не можу зрозуміти...
1,Я танцюю І в цьому танці я існую! Мої вуха Пер...
2,"Зранку, лиш розкрию очі, Через жінку перескочу..."
3,Увага! Увага! Небезпечна лаба! Що за?! Що за?!...
4,Від самого ранку весь офіс гудів – Фірма влашт...


In [10]:
train['Text'] = text_series[0]
test['Text'] = text_test_series[0]

In [11]:
test.head()

,song,dict,band,Text
0,001.txt,"{'artist': 'Тартак', 'title': 'Зима хвора'}",0,"Крижинки, як хмари – бачиш їх – з неба падають..."
1,002.txt,"{'artist': 'Тартак', 'title': 'Ілюзія'}",0,"У місті жорстокому, у місті розлюченому Я зажи..."
2,003.txt,"{'artist': 'Тартак', 'title': 'Я не знаю'}",0,"Я не знаю, що робити... Я не знаю, що казати....."
3,004.txt,"{'artist': 'Тартак', 'title': 'Майже на'}",0,"Ох, це вже нездолиме прагнення Нового здобуття..."
4,005.txt,"{'artist': 'Тартак', 'title': 'Нікому то не тр...",0,"Попадали на землю Всі ті, шо я придумав, слова..."


In [12]:
train.head()

,song,dict,band,Text
0,001.txt,"{'artist': 'Тартак', 'title': '100-ий плагіат'}",0,Дивлюсь на себе в дзеркало й не можу зрозуміти...
1,002.txt,"{'artist': 'Тартак', 'title': '3Dенс'}",0,Я танцюю І в цьому танці я існую! Мої вуха Пер...
2,003.txt,"{'artist': 'Тартак', 'title': 'DJ Вовочка'}",0,"Зранку, лиш розкрию очі, Через жінку перескочу..."
3,004.txt,"{'artist': 'Тартак', 'title': 'Mikpoffonha пер...",0,Увага! Увага! Небезпечна лаба! Що за?! Що за?!...
4,005.txt,"{'artist': 'Тартак', 'title': 'Qарпа-ратів'}",0,Від самого ранку весь офіс гудів – Фірма влашт...


# Text Preprocessing. Analysis

In [13]:
t = text_series.iloc[0][0]

In [14]:
#detecting language of the given sentence
text = TextBlob(t)
text.detect_language()

'uk'

In [15]:
# Let's define tokens for the given sentence
tokens_t = TextBlob(t)
tokens_t.words

WordList(['Дивлюсь', 'на', 'себе', 'в', 'дзеркало', 'й', 'не', 'можу', 'зрозуміти', 'Чому', 'я', 'такий', 'гарний', 'і', 'чому', 'такий', 'талановитий', 'Вигадую', 'мелодії', 'гармонії', 'чарівної', 'І', 'філософські', 'тексти', 'глибини', 'неймовірної', 'Я', 'хочу', 'щоб', 'мій', 'голос', 'усі', 'пізнавали', 'Мої', 'пісні', 'співали', 'усі', 'слова', 'напам', '’', 'ять', 'знали', 'Я', 'хочу', 'захлинутися', 'славою', 'негіркою', 'Я', 'хочу', 'стати', 'справжньою', 'супер-пупер-зіркою', 'Але', 'щоб', 'усе', 'це', 'мати', 'Треба', 'зовсім', 'небагато', 'Прокрутіть', 'цю', 'пісню', 'по', 'радіо', 'Прокрутіть', 'цю', 'пісню', 'по', 'радіо', 'О-о-о', 'Прокрутіть', 'цю', 'пісню', 'по', 'радіо', 'Прокрутіть', 'цю', 'пісню', 'по', 'радіо', 'О-о-о', 'Я', 'хочу', 'заробляти', 'великі-превеликі', 'гроші', 'Бо', 'хоч', 'вони', 'погано', 'пахнуть', 'але', 'такі', 'хороші', 'Кататися', 'по', 'місту', 'в', 'блискучому', 'довгому', 'лімузині', 'Й', 'купувати', 'речі', 'в', 'дорогому', 'магазині', 'Я'

In [16]:
# Split to sentences
tokens_t.sentences

[Sentence("Дивлюсь на себе в дзеркало й не можу зрозуміти, Чому я такий гарний і чому такий талановитий?"),
 Sentence("Вигадую мелодії гармонії чарівної І філософські тексти глибини неймовірної."),
 Sentence("Я хочу, щоб мій голос усі пізнавали, Мої пісні співали, усі слова напам’ять знали."),
 Sentence("Я хочу захлинутися славою негіркою, Я хочу стати справжньою супер-пупер-зіркою!"),
 Sentence("Але щоб усе це мати, Треба зовсім небагато!"),
 Sentence("Прокрутіть цю пісню по радіо!"),
 Sentence("Прокрутіть цю пісню по радіо!"),
 Sentence("О-о-о!"),
 Sentence("Прокрутіть цю пісню по радіо!"),
 Sentence("Прокрутіть цю пісню по радіо!"),
 Sentence("О-о-о!"),
 Sentence("Я хочу заробляти великі-превеликі гроші, Бо хоч вони погано пахнуть, але такі хороші!"),
 Sentence("Кататися по місту в блискучому довгому лімузині, Й купувати речі в дорогому магазині."),
 Sentence("Я хочу, щоб мене любили високі фотомоделі, Щоб з ними розважатися в п’ятизірковому готелі."),
 Sentence("Повісити на шию три

In [17]:
stop_words = get_stop_words('ukrainian')

In [18]:
stop_words

['a',
 'б',
 'в',
 'г',
 'е',
 'ж',
 'з',
 'м',
 'т',
 'у',
 'я',
 'є',
 'і',
 'аж',
 'ви',
 'де',
 'до',
 'за',
 'зі',
 'ми',
 'на',
 'не',
 'ну',
 'нх',
 'ні',
 'по',
 'та',
 'ти',
 'то',
 'ту',
 'ті',
 'це',
 'цю',
 'ця',
 'ці',
 'чи',
 'ще',
 'що',
 'як',
 'їй',
 'їм',
 'їх',
 'її',
 'або',
 'але',
 'ало',
 'без',
 'був',
 'вам',
 'вас',
 'ваш',
 'вже',
 'все',
 'всю',
 'вся',
 'від',
 'він',
 'два',
 'дві',
 'для',
 'ким',
 'мож',
 'моя',
 'моє',
 'мої',
 'міг',
 'між',
 'мій',
 'над',
 'нам',
 'нас',
 'наш',
 'нею',
 'неї',
 'них',
 'ніж',
 'ній',
 'ось',
 'при',
 'про',
 'під',
 'пір',
 'раз',
 'рік',
 'сам',
 'сих',
 'сім',
 'так',
 'там',
 'теж',
 'тим',
 'тих',
 'той',
 'тою',
 'три',
 'тут',
 'хоч',
 'хто',
 'цей',
 'цим',
 'цих',
 'час',
 'щоб',
 'яка',
 'які',
 'адже',
 'буде',
 'буду',
 'будь',
 'була',
 'були',
 'було',
 'бути',
 'вами',
 'ваша',
 'ваше',
 'ваші',
 'весь',
 'вниз',
 'вона',
 'вони',
 'воно',
 'всею',
 'всім',
 'всіх',
 'втім',
 'геть',
 'далі',
 'двох',


In [19]:
#Compute how many words in example sentence is NOT in the stopwords list:

def content_fraction(text):
    stopwords = get_stop_words('ukrainian')
    content = [w for w in text if w.lower() not in stopwords]
    return round((len(content) / len(text)), 4)
tok = nltk.word_tokenize(t)
print(content_fraction(tok)*100, '%')

66.14999999999999 %


# Feature engineering

## 1. Bag-of-Words Approach (CountVectorizer)

The more often a word/token appears in a document, the more important it is.

**PROS:** Easy to use and understand.
Built-in many scientific/NLP libraries.
Memory-efficient sparse format.
Works well enough.

**CONS:** 
Huge corpus usually leads to huge vocabulary size.
Doesn't catch details (semantics, relations, structure etc.).
Orderless representation.

### 1.1 TF-IDF (TfIdfVectorizer)

Bag-of-words approach that is slightly modified: 
If a word/token appears in a document, but rarely appears in other documents - it is important and vice versa: 
if its commonly across most documents - then we cannot rely on this word to help us distinquish between texts


**Mostly used Parameters:**
* **analyzer**={‘word’, ‘char’, ‘char_wb’} - what token to use (word, char-n-grams etc.)
* **ngram_range**=(min_n, max_n) - what N to use: say, ngram_range=(1,2) $\rightarrow$  use both unigrams and bigrams
* **stop_words**={‘english’, list_of_words, or None} (default) - whether to filter stop-words or not
* **max_features**={N, None} - to build a vocabulary that only consider the top N ordered by term frequency across the corpus
* **norm**={‘l1’, ‘l2’ or None, optional} - norm feature vector to unit norm ($L_2-$, $L_1-$ norms)

### 1.2 Hashes (HashingVectorizer)

This vectorizer implementation uses the hashing trick to find the mapping of token string name to feature integer index.

**PROS:** 
Very memory-scalable to large datasets as there is no need to store a vocabulary dictionary in memory

**CONS:**
There is no way to compute the inverse transform (to get from feature indices to string feature names) 
which can be a problem when trying to introspect which features are most important to a model.


**Mostly used Parameters:**
* **analyzer**={‘word’, ‘char’, ‘char_wb’} - what token to use (word, char-n-grams etc.)
* **ngram_range**=(min_n, max_n) - what N to use: say, ngram_range=(1,2) $\rightarrow$  use both unigrams and bigrams
* **stop_words**={‘english’, list_of_words, or None} (default) - whether to filter stop-words or not
* **n_features**={N} - how many "buckets" to use
* **norm**={‘l1’, ‘l2’ or None, optional} - norm feature vector to unit norm ($L_2-$, $L_1-$ norms)
* **non_negative**={True,False} whether to use non-negative values only (othervise, they will be centered around 0)

In [20]:
count_vectorizer = CountVectorizer(analyzer='word', 
                    ngram_range=(1,4),
                    vocabulary=None, # or vocabulary=your_own_dictionary
                    max_df=0.7, # don't filter words by their frequency
                    max_features=20 )

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 5),
                              analyzer = 'word', binary = True, max_df = 0.5, vocabulary=None)

tfidf_vectorizer_modified = TfidfVectorizer(ngram_range=(1, 5),
                              analyzer = 'char_wb', binary = True, max_df = 0.6, vocabulary=None)

hash_vectorizer = HashingVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,4)
                    )

### Summary for feature extraction:
#### from all vectorizers TF-idf is widely used for different tasks and after some modelling I saw that results for each model are best with this method. 

# Models

## Multinomial Naive Bayes

alpha - additive (Laplace/Lidstone) smoothing parameter

Naive Bayes is the simplest form of Bayesian network, in which all attributes are independent given the value of the class variable. This is called conditional independence. 


In [21]:
nb = MultinomialNB(alpha=0.000001) # almost no smoothing

pipeline_nb = Pipeline([('vectorizer', tfidf_vectorizer),
                     ('clf_svm', nb)])

# Cross-validation
scores_nb = cross_val_score(pipeline_nb, train.Text, train.band, cv=4, n_jobs=-1, scoring="roc_auc")
print('Cross validation accuracy on logistic regression:', scores_nb.mean())

Cross validation accuracy on logistic regression: 0.988561076605


In [22]:
pipeline_nb.fit(train["Text"],train["band"])

predicted_nb = pipeline_nb.predict_proba(test["Text"])[:,1]
y_pred_nb = [0 if pred < 0.5 else 1 for pred in predicted_nb]

In [23]:
print('Total test accuracy: ', str(accuracy_score(test['band'], y_pred_nb)*100)+'%')

Total test accuracy:  97.619047619%


## Logistic regression

In [24]:
# Check Logistic Regression
clf_logr = LogisticRegression(C=5)

pipeline_logr = Pipeline([('vectorizer', tfidf_vectorizer_modified),
                     ('clf_svm', clf_logr)])

# Cross-validation
scores_logr = cross_val_score(pipeline_logr, train.Text, train.band, cv=4, n_jobs=-1, scoring="roc_auc")
print('Cross validation accuracy on logistic regression:', scores_logr.mean())

Cross validation accuracy on logistic regression: 0.997412008282


In [25]:
pipeline_logr.fit(train["Text"],train["band"])

predicted_logr = pipeline_logr.predict_proba(test["Text"])[:,1]
y_pred_logr = [0 if pred < 0.5 else 1 for pred in predicted_logr]

In [26]:
print('Total test accuracy: ', str(accuracy_score(test['band'], y_pred_logr)*100)+'%')

Total test accuracy:  97.619047619%


## Linear SVM (Best results!)

In [27]:
# Create SVM model 
svm = LinearSVC(C=1)
clf_svm = CalibratedClassifierCV(svm) 

pipeline_svm = Pipeline([('vectorizer', tfidf_vectorizer_modified),
                     ('clf_svm', clf_svm)])

# Cross-validation
scores_svm = cross_val_score(pipeline_svm, train.Text, train.band, cv=4, n_jobs=-1, scoring="roc_auc")
print('Cross validation accuracy on svm:', scores_svm.mean())

Cross validation accuracy on svm: 0.997412008282


In [28]:
pipeline_svm.fit(train["Text"],train["band"])

predicted_svm = pipeline_svm.predict_proba(test["Text"])[:,1]
y_pred_svm = [0 if pred < 0.5 else 1 for pred in predicted_svm]

In [29]:
print('Total test accuracy: ', str(accuracy_score(test['band'], y_pred_svm)*100) + '%')

Total test accuracy:  100.0%


# Ensemble. RandomForest

I also tried RandomForest model to compare results achieved with linear modelling. 

At the end, Random Forest as an ensemble learning method gave much worse results than linear models.

In [30]:
def tokenize(text):
    text = TextBlob(text)
    return text.words

# split data on train and test 
X_train, X_val, y_train, y_val  = train_test_split(
        train['Text'], 
        train['band'],
        test_size=0.1, 
        random_state=42)

#transform text to feature vectors
train_hash = tfidf_vectorizer_modified.fit_transform(X_train)         #train data
valid_hash = tfidf_vectorizer_modified.transform(X_val)          #validation data
test_hash = tfidf_vectorizer_modified.transform(test['Text'])   #test data

In [31]:
clf_rf = RandomForestClassifier(n_estimators = 100, 
                                       min_samples_leaf=5, 
                                       random_state = 17,
                                       class_weight='balanced',
                                       verbose=1
                                      )
model_gbm = clf_rf.fit(X=train_hash, y=y_train)
y_pred_gbm = model_gbm.predict(test_hash)
print ("Test Accuracy: ", accuracy_score(test['band'], y_pred_gbm))

Test Accuracy:  0.833333333333


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


# Notes: 
* After using stopwords accuracy became ~7% worse. This is due to the dataset's size which is small. 
* In practical classification tasks, linear logistic regression and SVMs often give very similar results. Logistic regression tries to maximize the conditional likelihoods of the training data, which makes it more prone to outliers than SVMs. The SVMs mostly care about the points that are closest to the decision boundary (support vectors).

# Summary:
* Linear models work pretty good for NLP tasks, however, well tuned RNN will work better, but there is no reason to use neural network in this particular small task.
* Bag-of-words is very simple method of extracting features, still gives good results. 
* After testing and tuning several vectorizors I stopped at TF-idf approach as it provided best accuracy on both validation and test data.
